In [1]:
import pandas as pd

import numpy as np


jb = pd.read_csv('datasets/2020-jetbrains-python-survey.csv')
print(jb)

C:\Users\Timofey\AppData\Local\Temp\ipykernel_13172\587314385.py:6: DtypeWarning: Columns (152) have mixed types. Specify dtype option on import or set low_memory=False.
  jb = pd.read_csv('datasets/2020-jetbrains-python-survey.csv')


      is.python.main other.lang.None other.lang.Java other.lang.JavaScript  \
0                Yes             NaN             NaN                   NaN   
1                Yes             NaN            Java            JavaScript   
2                Yes             NaN             NaN                   NaN   
3                Yes             NaN             NaN            JavaScript   
4                Yes             NaN            Java            JavaScript   
...              ...             ...             ...                   ...   
54457            Yes             NaN             NaN                   NaN   
54458            Yes             NaN             NaN            JavaScript   
54459            Yes             NaN             NaN            JavaScript   
54460            Yes             NaN             NaN            JavaScript   
54461            Yes             NaN             NaN            JavaScript   

      other.lang.C/C++ other.lang.PHP other.lang.C# other.lang.

1. Create a ”tweak” function to clean up the data

In [4]:
import catboost as cb
import collections

def get_unique_cols(jb):
    counter = collections.defaultdict(list)
    for col in sorted(jb.columns):
        period_count = col.count('.')
        if period_count >= 2:
            part_end = 2
        else:
            part_end = 1
        parts = col.split('.')[:part_end]
        counter['.'.join(parts)].append(col)
    unique_cols = []
    for cols in counter.values():
        if len(cols) == 1:
                unique_cols.extend(cols)
    return unique_cols

def prep_for_ml(df):
    # remove pandas types
    return (df
           .assign(**{col:df[col].astype(float)
                     for col in df.select_dtypes('number')},
                   **{col:df[col].astype('str').fillna('')
                     for col in df.select_dtypes(['object', 'category'])})
           )

def predict_col(df, col):
    df = prep_for_ml(df)
    missing = df.query(f'~{col}.isna()')
    cat_idx = []
    for i, typ in enumerate(df.drop(columns=[col]).dtypes):
        if str(typ) == 'object':
            cat_idx.append(i)
    X = missing.drop(columns=[col]).values
    y = missing[col]
    model = cb.CatBoostRegressor(iterations=20, cat_features=cat_idx)
    model.fit(X, y, cat_features=cat_idx)
    pred = model.predict(df.drop(columns=[col]))
    return df[col].where(~df[col].isna(), pred)

def tweak_jb(jb):
    unique_cols = get_unique_cols(jb)
    return (jb
            [unique_cols]
           .rename(columns=lambda c: c.replace('.', '_'))
           .assign(age=lambda df_: df_.age.str.slice(0, 2).astype(float)
                  .astype('Int64'),
                  are_you_datascientist=lambda df_: df_
                      .are_you_datascientist
                      .replace({'Yes': True, 'No': False, np.nan: False}),
                 # company_size=lambda df_: df_.company_size.replace({
                  #    'Just me': 1, 'Not sure': np.nan,  
                   #   'More than 5,000': 5000, '2-10': 2, '11-50': 11, ?!?!?!?!?!?!?!? what's wrong?
                    #  '51-500': 51, '501-1,000': 501,
                     # '1,001-5,001': 1001}).astype('Int64'),
                  country_live=lambda df_: df_.country_live
                      .astype('category'),
                  employment_status=lambda df_: df_.employment_status
                      .fillna('Other').astype('category'),
                  is_python_main=lambda df_: df_.is_python_main
                      .astype('category'),
                  team_size=lambda df_: df_.team_size
                      .str.split(r'-', n=1, expand=True)
                      .iloc[:, 0].replace('More than 40 people', 41)
                      .where(df_.company_size!=1, 1).astype(float),
                  years_of_coding=lambda df_: df_.years_of_coding
                      .replace('Less than 1 year', .5)
                      .str.extract(r'(\d+)').astype(float),
                  python_years=lambda df_: df_.python_years
                      .replace('Less than 1 year', .5)
                      .str.extract(r'(\d+)').astype(float),
                  python3_ver=lambda df_: df_.python3_version_most
                      .str.replace('_', '.').str.extract(r'(\d\.\d)')
                      .astype(float),
                  use_python_most=lambda df_: df_.use_python_most
                      .fillna('Unknown')
                  )
           .assign(team_size=lambda df_:predict_col(df_, 'team_size')
                  .astype(int))
           .drop(columns=['python2_version_most'])
           .dropna()
        )

jb2 = tweak_jb(jb)
print(jb2)

Learning rate set to 0.5
0:	learn: 6.3373204	total: 170ms	remaining: 3.23s
1:	learn: 6.2416428	total: 195ms	remaining: 1.75s
2:	learn: 6.2042819	total: 220ms	remaining: 1.25s
3:	learn: 6.1713242	total: 247ms	remaining: 987ms
4:	learn: 6.1313697	total: 271ms	remaining: 812ms
5:	learn: 6.1163744	total: 295ms	remaining: 688ms
6:	learn: 6.1054688	total: 318ms	remaining: 590ms
7:	learn: 6.1040920	total: 337ms	remaining: 506ms
8:	learn: 6.0946976	total: 363ms	remaining: 444ms
9:	learn: 6.0906277	total: 379ms	remaining: 379ms
10:	learn: 6.0894286	total: 401ms	remaining: 328ms
11:	learn: 6.0757775	total: 425ms	remaining: 284ms
12:	learn: 6.0718265	total: 451ms	remaining: 243ms
13:	learn: 6.0696191	total: 475ms	remaining: 204ms
14:	learn: 6.0644258	total: 498ms	remaining: 166ms
15:	learn: 6.0638403	total: 510ms	remaining: 128ms
16:	learn: 6.0469462	total: 533ms	remaining: 94.1ms
17:	learn: 6.0442746	total: 555ms	remaining: 61.7ms
18:	learn: 6.0442742	total: 565ms	remaining: 29.7ms
19:	learn: 6.

2. Explore the memory usage of the raw data and the tweaked data

In [7]:
print(jb.memory_usage(deep=True))
print(jb2.memory_usage(deep=True))

Index                           128
is.python.main              3385152
other.lang.None             1928442
other.lang.Java             1977945
other.lang.JavaScript       2325954
                             ...   
job.role.CIO / CEO / CTO    1795584
job.role.Systems analyst    1797304
job.role.Other              1820634
age                         3214332
country.live                2576032
Length: 265, dtype: int64
Index                          112232
age                            126261
are_you_datascientist          477016
company_size                  1332821
country_live                    21082
employment_status               15095
first_learn_about_main_ide    1067634
how_often_use_main_ide         871773
ide_main                      1006091
is_python_main                  14468
job_team                      1139942
main_purposes                 1120716
missing_features_main_ide     1449631
nps_main_ide                   112232
python_years                   112232
python3_